## A 08 
文本抄袭的判断 --- 找出判断为新华社但并非新华社的文章
  + 导入数据 & 文本预处理;
  + 文本表征（Tfidf）;
  + 划分训练集和测试集;
  + 训练分类器;
  + 验证性能 (precision, recall);
  + 参数调整
  + 保存和再使用

In [1]:
!pwd

/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/nlp-AI-course/L08


In [2]:
import pandas as pd

file_path = '/Users/xinweixu/Dropbox/learn/Comp_Prog/nlp/data/sqlResult_1558435.csv'

In [3]:
content = pd.read_csv(file_path, encoding = 'gb18030')

In [5]:
content.head()

,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [6]:
content = content.fillna('')

In [7]:
# take a look at all articles with sources coming from xinhua

content[content['source']=='新华社']

,id,author,source,content,feature,title,url
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
39,89578,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....
95,89522,,新华社,新华社韩国济州6月18日电综述：亚投行第二届年会三大亮点\r\n新华社记者 耿学鹏 严蕾\r...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",综述：亚投行第二届年会三大亮点,http://china.huanqiu.com/hot/2017-06/10866439....
113,89504,,新华社,新华社北京6月18日电 经军委领导批准，《军营理论热点怎么看·2017》日前印发全军。\r\...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",《军营理论热点怎么看·2017》印发全军,http://china.huanqiu.com/hot/2017-06/10866447....
305,89312,,新华社,新华社兰州6月19日电（记者张钦）记者19日了解到，刚刚出台的《甘肃省网络扶贫行动的实施方案...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",甘肃力推网络扶贫：贫困山区到2020年实现4G网络全覆盖,http://china.huanqiu.com/hot/2017-06/10866626....
312,89305,,新华社,新华社乌鲁木齐6月18日电（记者杜刚、王秉阳）6月18日，新疆天山天池首次举办山地马拉松比赛...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",天山天池首次举办山地马拉松赛,http://china.huanqiu.com/hot/2017-06/10866627....
314,89303,,新华社,新华社北京6月19日电（记者熊丰）记者从公安部消防局获悉，19日上午6时，由公安部消防局组织...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",9省市联动开展跨区域地震救援实战拉动演练,http://china.huanqiu.com/hot/2017-06/10866630....
543,89074,刘瑞莹,新华社,暑期旅游旺季是各大旅游景区开门迎客的好时机。然而，有5家5A级景区却高兴不起来：国家旅游...,"{""type"":""旅游"",""site"":""环球"",""commentNum"":""0"",""joi...",一年内全国12家5A级景区被国家旅游局摘牌或警告,http://go.huanqiu.com/news/2016-10/9542454.html
545,89072,,新华社,新华社北京6月20日电 总部设在德国波恩的国际残奥委会在当地时间19日宣布，中国残联主席...,"{""type"":""体育"",""site"":""新浪"",""commentNum"":"" 50"",""j...",张海迪获得国际残奥委会主席参选提名,http://sports.sina.com.cn/others/others/2017-0...
651,88966,王点,新华社,新华社南京4月17日电(记者潘晔)江苏拥有不少5A景区。日前，一份来自清华大学的第三方调...,"{""type"":""旅游"",""site"":""环球"",""commentNum"":""0"",""joi...",拿什么消除游客“差评”？—江苏旅游业调查,http://go.huanqiu.com/news/2017-04/10493712.html


In [8]:
content[content['source'].str.contains('新华')]

,id,author,source,content,feature,title,url
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
17,89600,,新华网,国歌法草案首次提请最高立法机关审议\r\n点击图片进入下一页\r\n新闻：十二届全国人大常委...,"{""type"":""观点滚动"",""site"":""参考消息"",""commentNum"":""0"",...",[及时点]我们不需要“百变老太太”,http://column.cankaoxiaoxi.com/plgd/2017/0623/...
25,89592,,新华网,当地时间2017年6月17日，土耳其马尼萨省，在土耳其第1步兵训练大队司令部军营，590名土...,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",土耳其近590名士兵疑食物中毒 被送往医院救治（组图）,http://world.huanqiu.com/hot/2017-06/10866176....
27,89590,,新华网,英国与欧盟的“脱欧”谈判于19日正式开始。然而此时，英国首相特雷莎·梅正面临着空前的政治压力...,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",火灾与脱欧炙烤英国首相 特雷莎·梅面临空前政治压力,http://world.huanqiu.com/hot/2017-06/10866198....
39,89578,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态,http://world.huanqiu.com/hot/2017-06/10866229....
51,89566,,新华网,戈壁的大漠黄沙曾掩埋了无数西域古道，而如今一条大漠天路正顽强地与黄沙“搏斗”，在乌兰布和、腾...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",大漠变通途——世界上最长的穿越沙漠高速公路建设纪实,http://china.huanqiu.com/hot/2017-06/10866392....
65,89552,,新华社@,从百度、支付宝到App?“高考志愿填报”五花八门\r\n顾女士近来减少了刷朋友圈的频率，一有...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""5"",...",高考志愿填报：眼花缭乱的App、上万元咨询靠谱吗？,http://www.cnbeta.com/articles/tech/623637.htm
95,89522,,新华社,新华社韩国济州6月18日电综述：亚投行第二届年会三大亮点\r\n新华社记者 耿学鹏 严蕾\r...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",综述：亚投行第二届年会三大亮点,http://china.huanqiu.com/hot/2017-06/10866439....
104,89513,,新华网,粤港澳概念板块午后走强，珠海港、恒基达鑫、白云机场等个股的午后纷纷快速拉升。\r\n据悉，首...,"{""type"":""金融市场"",""site"":""参考消息"",""commentNum"":""0"",...",粤港澳概念板块午后走强 珠海港等个股领涨,http://www.cankaoxiaoxi.com/finance/20170623/2...
113,89504,,新华社,新华社北京6月18日电 经军委领导批准，《军营理论热点怎么看·2017》日前印发全军。\r\...,"{""type"":""国内新闻"",""site"":""环球"",""commentNum"":""0"",""j...",《军营理论热点怎么看·2017》印发全军,http://china.huanqiu.com/hot/2017-06/10866447....


In [9]:
import numpy as np

In [10]:
# create a flag variable, is_xinhua, indicating whether the article comes from xinhua

content['is_xinhua'] = np.where(content['source'].str.contains('新华'), 1, 0)

In [11]:
content.head()

,id,author,source,content,feature,title,url,is_xinhua
0,89617,,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm,0
1,89616,,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm,0
2,89615,,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm,0
3,89614,,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....,1
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...,0


In [12]:
content['is_xinhua'][:10]

0    0
1    0
2    0
3    1
4    0
5    0
6    0
7    0
8    0
9    0
Name: is_xinhua, dtype: int64

In [13]:
import jieba

def cut(string): return ' '.join(jieba.cut(string))

In [14]:
cut('今天天气真好')

Building prefix dict from the default dictionary ...
Dumping model to file cache /var/folders/1y/1btp7xpj7b1f82lnwvn2916h0000gn/T/jieba.cache
Loading model cost 0.797 seconds.
Prefix dict has been built succesfully.


'今天天气 真 好'

In [15]:
content['title'] = content['title'].fillna('')

In [16]:
help(content.apply)

Help on method apply in module pandas.core.frame:

apply(func, axis=0, broadcast=None, raw=False, reduce=None, result_type=None, args=(), **kwds) method of pandas.core.frame.DataFrame instance
    Apply a function along an axis of the DataFrame.
    
    Objects passed to the function are Series objects whose index is
    either the DataFrame's index (``axis=0``) or the DataFrame's columns
    (``axis=1``). By default (``result_type=None``), the final return type
    is inferred from the return type of the applied function. Otherwise,
    it depends on the `result_type` argument.
    
    Parameters
    ----------
    func : function
        Function to apply to each column or row.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis along which the function is applied:
    
        * 0 or 'index': apply function to each column.
        * 1 or 'columns': apply function to each row.
    broadcast : bool, optional
        Only relevant for aggregation functions:
    
       

In [17]:
# cut the article title into tokens

content['title'] = content['title'].apply(cut)

In [18]:
content['title'][:10]

0                  小米 MIUI   9 首批 机型 曝光 ： 共计 15 款
1           骁龙 835 在 Windows   10 上 的 性能 表现 有望 改善
2           一加 手机 5 细节 曝光 ： 3300mAh 、 充 半小时 用 1 天
3    葡 森林 火灾 造成 至少 62 人 死亡   政府 宣布 进入 紧急状态 （ 组图 ）
4         44 岁 女子 约 网友 被 拒 暴雨 中 裸奔   交警 为 其 披衣 相随
5                             金融股 一枝独秀   配置 价值 犹存
6                    配骁龙 660   全面 屏 鼻祖 夏普 新机 酝酿 中
7                          博友 早评 ： 震荡 有利 消化 套牢 筹码
8              武汉 千余 警察 出动   抓获 808 名 俊男靓女 全是 诈骗犯
9                     纳入 MSCI 指数   A股 长期 配置 价值 提升
Name: title, dtype: object

In [19]:
len(content)

89611

In [20]:
# divide into training set and test set
# to speed up the process, we first try on a subsample

content_sample = content.sample(n=5000)

In [21]:
len(content_sample)

5000

In [22]:
# find all xinhua news contained in the sample

xinhua_news = content_sample[content_sample['source'].str.contains('新华')]

In [23]:
len(xinhua_news)

4427

In [24]:
# percentage of xinhua news in the sample

len(xinhua_news)/len(content_sample)

0.8854

In [25]:
# build a classfier only using the article titles
# get the values for the predictor, the flag variable, is_xinhua

y = content_sample['is_xinhua'].values

In [26]:
y[:10]

array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0])

In [27]:
# get the article titles

x_title = content_sample['title'].values

In [28]:
x_title[:10]

array(['[ 5 ] （ 外代 二线 ） 彩色 烟雾',
       '[ 9 ] （ 外代 二线 ） 网球 — — 法网 ： 巴辛 斯基 晋级 半决赛',
       '（ 体育 ） （ 9 ） 足球 — — 中超 ： 河南建业 平 江苏 苏宁 易购', '意大利 粪便 博物馆 变废为宝',
       '（ 体育 ） （ 20 ） 篮球 — — CBA 总决赛 ： 新疆 队 夺冠',
       '[ 6 ] （ 外代 二线 ） 篮球 — — NBA ： 湖人胜 马刺',
       '祝贺 丁宁 ！ 祝贺 中国 女乒 ！ 祝贺 孔 教练 ！ 不管 经历 什么 ， 我 眼里 的 老孔 ， 永远 是 当年 那个 亲吻 国旗 的 热血青年 。 他 永远 是 我 心中 的 英雄 。   ? ? ? ?',
       '图文 ： 铭记 历史 不忘 初心', '（ 教育 ） （ 9 ） 2017 年 全国 高考 报名 考生 共 940 万人',
       '美媒 ： 动物 实验 表明 锻炼 或能 减少 骨髓 脂肪'], dtype=object)

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=30000)
x_title = vectorizer.fit_transform(x_title)

In [30]:
x_title.shape

(5000, 10595)

In [31]:
# to randomly split the subsample into training and test set,
# we create an index to be shuffled

indices = np.arange(len(y))

In [32]:
np.random.shuffle(indices)

In [33]:
indices

array([4970, 3218,  433, ..., 1799, 1204,  984])

In [34]:
split_test = 0.25

In [35]:
# get the indices for the training and test sets
# --  the last 1/4 of the articles will be the training set
#     while the first 3/4 will be the test set

train_indices = indices[int(len(indices)*split_test):]
test_indices = indices[:int(len(indices)*split_test)]

In [36]:
# get the data using the indices
x_title_train, x_title_test = x_title[train_indices], x_title[test_indices]
y_train, y_test = y[train_indices], y[test_indices]

In [37]:
x_title_train

<3750x10595 sparse matrix of type '<class 'numpy.float64'>'
	with 27159 stored elements in Compressed Sparse Row format>

In [38]:
# 1) build a simple Naïve Bayesian classifier

from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [39]:
clf.fit(x_title_train.toarray(), y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [41]:
# define a procedure to evaluate the performance of the classifier

from sklearn.metrics import precision_score, recall_score, roc_auc_score, confusion_matrix

def get_performance(clf, x_, y_):
    """
    @clf: the model
    @x_: test data for the X vector
    @y_: test data for the y vector
    """
    y_hat = clf.predict(x_.toarray()) # get the predicted value of y
    print('precision is: {}'.format(precision_score(y_, y_hat)))
    print('recall is: {}'.format(recall_score(y_, y_hat)))
    print('roc_auc is: {}'.format(roc_auc_score(y_, y_hat)))
    print('confusion matrix: \n{}'.format(confusion_matrix(y_, y_hat, labels=[0, 1])))

In [42]:
get_performance(clf, x_title_train, y_train)

precision is: 1.0
recall is: 0.99789536981359
roc_auc is: 0.998947684906795
confusion matrix: 
[[ 424    0]
 [   7 3319]]


In [43]:
get_performance(clf, x_title_test, y_test)

precision is: 0.9351111111111111
recall is: 0.9554950045413261
roc_auc is: 0.7327810593176428
confusion matrix: 
[[  76   73]
 [  49 1052]]


In [44]:
# 2) build a decision tree classifier

from sklearn.tree import DecisionTreeClassifier
clf_tree = DecisionTreeClassifier(class_weight={0:5, 1:4}, criterion='entropy',
                                 max_features=5000)

In [46]:
clf_tree.fit(x_title_train, y_train)

DecisionTreeClassifier(class_weight={0: 5, 1: 4}, criterion='entropy',
                       max_depth=None, max_features=5000, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [47]:
get_performance(clf_tree,x_title_train,y_train)

precision is: 1.0
recall is: 1.0
roc_auc is: 1.0
confusion matrix: 
[[ 424    0]
 [   0 3326]]


In [48]:
get_performance(clf_tree,x_title_test,y_test)

precision is: 0.9686013320647003
recall is: 0.924613987284287
roc_auc is: 0.8515687386085865
confusion matrix: 
[[ 116   33]
 [  83 1018]]


In [49]:
y_hat_test = clf.predict(x_title_test.toarray())

In [51]:
len(y_hat_test)

1250

In [52]:
len(y_test)

1250

In [53]:
candidate_indices = []
for index, (y, yhat) in enumerate(zip(y_test,y_hat_test)):
    if y == 0 and yhat == 1:
        candidate_indices.append(test_indices[index])

In [54]:
len(candidate_indices)

73

In [55]:
candidate_indices

[331,
 1683,
 2412,
 2809,
 1788,
 218,
 4282,
 4017,
 3016,
 4563,
 4794,
 601,
 919,
 3590,
 469,
 2422,
 4362,
 3840,
 4099,
 557,
 4120,
 3272,
 985,
 1881,
 4327,
 3864,
 4040,
 4304,
 4615,
 4715,
 3912,
 529,
 2150,
 2962,
 4817,
 4591,
 2739,
 3764,
 4006,
 2654,
 4339,
 1186,
 3848,
 816,
 1549,
 2013,
 4306,
 2507,
 109,
 588,
 632,
 1743,
 609,
 2586,
 3937,
 1116,
 1235,
 374,
 4579,
 3219,
 3037,
 3115,
 1810,
 4455,
 2297,
 550,
 779,
 3376,
 1611,
 2641,
 4753,
 84,
 1519]

In [56]:
content.iloc[331]['content']

'新华社北京６月２３日电（记者倪元锦）２１日起，京津冀迎来今年以来最强降雨。记者采访气象专家，解读“此轮最强降雨有何特点”“华北汛期强降雨趋势怎样”“在防汛抗洪方面面临哪些压力”等公众关注的问题。\r\n京津冀今年以来“最强降雨”有何特点？\r\n北京市气象台发布２１日１２时至２３日１３时降水量显示，全市平均７２．７毫米，城区平均７９．８毫米，最大怀柔九渡河１８５．０毫米。截至１３时，北京处在暴雨黄色预警、地质灾害黄色预警中，预计２３日傍晚局地有短时大风或冰雹，２４日仍有降水。\r\n北京市气象局首席预报员熊亚军说，京津冀地区正迎来今年最强降水，其特点是持续时间长、累积雨量大、影响范围广，同时会带来局地强对流天气。对北京而言，此轮降水将达“暴雨”量级，预计累计降水量为６０毫米－１００毫米，局地累计降水量１２０毫米－１８０毫米，最大小时雨强为４０毫米－６０毫米。\r\n“今年以来，北京全市降水量仅６２毫米，本轮降水很可能超过上半年的降水量。从累计雨量判断，它的确将是今年以来北京最强的降雨过程。”熊亚军说。\r\n“华北这次降雨过程初期对流活动强、中期平缓、后期有所加强。具体而言，前期和后期对流活动明显，打雷又下雨，中间趋于稳定，累计雨量不小。要谨防降雨过程中可能出现的强对流天气。”熊亚军说。\r\n此轮京津冀降雨的成因是什么？\r\n气象专家介绍，从科学成因上讲，这次降雨过程由“低涡”和“冷涡”系统所致。\r\n据介绍，“冷涡”是指存在于中、高纬度地区，对流层中、上层的冷性的闭合低压环流系统，其中心温度明显低于四周；“低涡”常指出现于大气中的气旋式低压涡旋，在低涡区内常伴有较强的上升运动，假如水汽充沛，大气又呈不稳定状态，则低涡也可能产生暴雨。\r\n“当前，低涡的位置和强度直接影响此次降雨的落区和强度。而目前来看，低涡系统的变化仍有不确定性，因此使预报难度增加。”熊亚军说。\r\n华北汛期强降雨趋势怎样\r\n根据国家气候中心会商意见，今年６－８月全国降水总体“北少南多”，华北地区降水量总体比常年同期偏少，但京津冀东北部６－８月降水量偏多１－２成，其他地区接近常年同期。\r\n'

In [57]:
content.iloc[331]['source']

'新华网'

In [58]:
content.iloc[1519]['content']

'中国经济网北京6月22日讯\u3000(记者韩秉志)在22日召开的全国“互联网+人社”推进座谈会上，人力资源和社会保障部负责人表示，要加强基于大数据的就业形势分析，开展更为精准的网上就业服务。\r\n人力资源和社会保障部部长尹蔚民在会上指出，实施“互联网+人社”是实现决策科学化、管理精准化、服务人本化的内在要求。关键是要积极推动各项业务的跨界，促进系统内、部门间及与社会机构的融合，构建分享、互联、众创、包容、协作的开放体系。\r\n尹蔚民说，实施“互联网+人社”，要重点加强基于大数据的就业形势分析、开展更为精准的网上就业服务、促进全民参保计划实施、实现异地就医结算、推进社保基层智能监控、深化职业资格信息化管理、推行劳动保障智慧监察、提高劳动人事争议处理信息化水平、努力实现让群众“最多跑一次”，通过“互联网+”对人社领域各项业务进行重塑和再造，不断提升人民群众的获得感和幸福感。\r\n'

In [59]:
content.iloc[1519]['source']

'中国经济网'

In [60]:
import pickle

In [61]:
# store the model

with open('clf', 'wb') as picklefile:
    pickle.dump(clf, picklefile)

In [62]:
# reload the model

with open('clf', 'rb') as training_model:
    model = pickle.load(training_model)

In [64]:
y_hat_test = model.predict(x_title_test.toarray())

In [65]:
len(y_hat_test)

1250

In [66]:
y_hat_test[:100]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])